# 隊名: TEAM_5729

In [1]:
# from google.colab import drive
# drive.mount("/content/drive")

# import os
# os.chdir('/content/drive/MyDrive/根據區域微氣候資料預測發電量競賽')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [3]:
# 讀取 CSV 檔
add = pd.read_csv('ADD_FEATURE.csv')

# 對 Face 欄位進行 One-Hot 編碼
face_encoded_df = pd.get_dummies(add['Face'], drop_first=True, prefix='Face')

# 將新欄位與原始 DataFrame 合併並刪除舊欄位
add = pd.concat([add.drop('Face', axis=1), face_encoded_df], axis=1)
add.head()

,LocationCode,Latitude,Longitude,Height,Face_Northwest,Face_South,Face_Southeast,Face_Southwest,Face_West
0,1,23.8994,121.5444,5,False,True,False,False,False
1,2,23.8997,121.5447,5,False,True,False,False,False
2,3,23.8997,121.5450,5,False,True,False,False,False
3,4,23.8994,121.5444,5,False,True,False,False,False
4,5,23.8994,121.5447,5,False,False,False,True,False


In [4]:
upload = pd.read_csv('upload(no answer).csv')

upload['year'] = upload['序號'].astype(str).str[:4].astype(int)          # 年份
upload['month'] = upload['序號'].astype(str).str[4:6].astype(int)        # 月份
upload['day'] = upload['序號'].astype(str).str[6:8].astype(int)          # 日期
upload['hour'] = upload['序號'].astype(str).str[8:10].astype(int)        # 小時
upload['minute'] = upload['序號'].astype(str).str[10:12].astype(int)     # 分鐘
upload['LocationCode'] = upload['序號'].astype(str).str[12:].astype(int) # 裝置代碼
upload['date'] = pd.to_datetime(upload[['year', 'month', 'day']])        # 日期
upload['weekday'] = upload['date'].dt.weekday                            # 星期幾 (0 表示星期一，6表示星期日）
upload = upload.drop(columns=['date'])
upload = pd.merge(upload, add, on='LocationCode', how='left')
upload = upload.rename(columns={'序號': 'Serial'})
upload

,Serial,答案,year,month,day,hour,minute,LocationCode,weekday,Latitude,Longitude,Height,Face_Northwest,Face_South,Face_Southeast,Face_Southwest,Face_West
0,20240117090001,NaN,2024,1,17,9,0,1,2,23.8994,121.5444,5,False,True,False,False,False
1,20240117091001,NaN,2024,1,17,9,10,1,2,23.8994,121.5444,5,False,True,False,False,False
2,20240117092001,NaN,2024,1,17,9,20,1,2,23.8994,121.5444,5,False,True,False,False,False
3,20240117093001,NaN,2024,1,17,9,30,1,2,23.8994,121.5444,5,False,True,False,False,False
4,20240117094001,NaN,2024,1,17,9,40,1,2,23.8994,121.5444,5,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,20241002161012,NaN,2024,10,2,16,10,12,2,23.8997,121.5447,1,True,False,False,False,False
9596,20241002162012,NaN,2024,10,2,16,20,12,2,23.8997,121.5447,1,True,False,False,False,False
9597,20241002163012,NaN,2024,10,2,16,30,12,2,23.8997,121.5447,1,True,False,False,False,False
9598,20241002164012,NaN,2024,10,2,16,40,12,2,23.8997,121.5447,1,True,False,False,False,False


In [5]:
# 季節劃分函數
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

upload['Season'] = upload['month'].apply(get_season)

# 對 Season 欄位進行 One-Hot 編碼
upload = pd.get_dummies(upload, columns=['Season'], drop_first=False)

In [6]:
upload[upload.select_dtypes(include='bool').columns] = upload.select_dtypes(include='bool').astype(int)

In [7]:
upload.columns

Index(['Serial', '答案', 'year', 'month', 'day', 'hour', 'minute',
       'LocationCode', 'weekday', 'Latitude', 'Longitude', 'Height',
       'Face_Northwest', 'Face_South', 'Face_Southeast', 'Face_Southwest',
       'Face_West', 'Season_Autumn', 'Season_Spring', 'Season_Summer',
       'Season_Winter'],
      dtype='object')

# Stacking Test

In [8]:
#  Stacking Test
from joblib import dump, load

scaler = load('scaler.joblib')  
loaded_stacking_model = load('stacking_model.joblib')  

new_data = upload[['Serial', 'month', 'day', 'hour', 'minute', 'LocationCode', 'weekday','Latitude', 'Longitude', 'Height', 'Face_South', 'Face_Southeast',
                   'Season_Winter', 'Season_Summer', 'Season_Spring']] # 使用特徵與訓練stacking_model時相同


# 使用加載的標準化器對新數據進行標準化
new_data_scaled = scaler.transform(new_data)

# 使用加載的 Stacking 模型進行預測
predictions = loaded_stacking_model.predict(new_data_scaled)
predictions[predictions < 0] = 0
upload['答案'] = np.round(predictions, 2)
print(np.round(predictions, 2))
upload.to_csv("upload_with_predictions.csv", index=False)

[304.15 332.71 408.4  ...  10.35  12.2    9.08]


In [9]:
upload_revise = pd.read_csv("upload_with_predictions.csv")
upload_revise = upload_revise.rename(columns={'Serial': '序號'})
upload_revise = upload_revise[['序號', '答案']]
upload_revise

,序號,答案
0,20240117090001,304.15
1,20240117091001,332.71
2,20240117092001,408.40
3,20240117093001,389.98
4,20240117094001,397.36
...,...,...
9595,20241002161012,4.96
9596,20241002162012,7.66
9597,20241002163012,10.35
9598,20241002164012,12.20


In [10]:
# 輸出要Submission的csv
upload_revise.to_csv('upload_revise_no_features.csv', index=False)